<a href="https://colab.research.google.com/github/zukol/dw_matrix_road_sign/blob/master/dw_matrix3_day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install hyperopt

In [27]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import datetime

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Conv1D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from skimage import color, exposure
from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [28]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/data'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/data


In [0]:
train = pd.read_pickle('train.p')
test = pd.read_pickle('test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1:
  y_train = to_categorical(y_train)

if y_test.ndim == 1:
  y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_calsses = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(X_train,
            y_train,
            batch_size = params_fit.get('batch_size', 128),
            epochs = params_fit.get('epochs', 5),
            verbose = params_fit.get('verbose', 1),
            validation_data = params_fit.get('validation_data', (X_train, y_train)),
            callbacks = [tensorboard_callback])
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [33]:
def get_cnn_v5(input_shape, num_calsses):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_calsses, activation='softmax')
])

#get_cnn_v5(input_shape, num_calsses).summary()
train_and_predict(get_cnn_v5(input_shape, num_calsses))

Epoch 1/5
272/272 [==============================] - 17s 63ms/step - loss: 2.2988 - accuracy: 0.3556 - val_loss: 0.6796 - val_accuracy: 0.7796
Epoch 2/5
272/272 [==============================] - 13s 50ms/step - loss: 0.6865 - accuracy: 0.7782 - val_loss: 0.1773 - val_accuracy: 0.9475
Epoch 3/5
272/272 [==============================] - 13s 49ms/step - loss: 0.3495 - accuracy: 0.8879 - val_loss: 0.0709 - val_accuracy: 0.9792
Epoch 4/5
272/272 [==============================] - 13s 50ms/step - loss: 0.2243 - accuracy: 0.9289 - val_loss: 0.0541 - val_accuracy: 0.9841
Epoch 5/5
272/272 [==============================] - 13s 49ms/step - loss: 0.1942 - accuracy: 0.9428 - val_loss: 0.0380 - val_accuracy: 0.9886


0.9639455782312926

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_calsses, activation='softmax')
])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(X_train,
            y_train,
            batch_size = int(params.get('batch_size', 128)),
            epochs = 5,
            verbose = 0
  )

  accuracy = model.evaluate(X_test, y_test)[0]
  print (params)#, 'accuracy={}'.format(accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK, 'model':model}

In [46]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.2371 - accuracy: 0.9062

  7/138 [>.............................]
 - ETA: 1s - loss: 0.1199 - accuracy: 0.9598

 12/138 [=>............................]
 - ETA: 1s - loss: 0.1271 - accuracy: 0.9531

 18/138 [==>...........................]
 - ETA: 1s - loss: 0.2559 - accuracy: 0.9323

 24/138 [====>.........................]
 - ETA: 1s - loss: 0.3048 - accuracy: 0.9193

 29/138 [=====>........................]
 - ETA: 1s - loss: 0.3125 - accuracy: 0.9030
